In [1]:
import os
import copy
from PIL import Image
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils import data
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.optim as optim
from PIL import Image
import numpy as np
from torchvision.datasets import ImageFolder

from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
augment_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224), antialias=True),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2), antialias=True),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5)], p=0.5),
    transforms.RandomApply([transforms.RandomAffine(0, translate=(0.2, 0.2))], p=0.5),
    transforms.RandomApply([transforms.RandomRotation(10)], p=0.5),
    transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), antialias=True),
    # transforms.Grayscale(num_output_channels=1)
    transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
])

In [3]:
path = "/kaggle/input/fer2013"
train_dataset = ImageFolder(path+"/train", transform=transform)
augment_dataset = ImageFolder(path+"/train", transform=augment_transform)
train_dataset = ConcatDataset([train_dataset, augment_dataset])
test_dataset = ImageFolder(path+"/test", transform=transform)

In [4]:
print(len(train_dataset))
print(len(test_dataset))
print(test_dataset[2][0].size())

57418
7178
torch.Size([3, 224, 224])


In [5]:
VALID_RATIO = 0.9
n_train_examples = int(len(train_dataset) * VALID_RATIO)
n_valid_examples = len(train_dataset) - n_train_examples
train_data, valid_data = data.random_split(train_dataset, [n_train_examples, n_valid_examples])
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = transform

In [6]:
def collate_fn(dataset):
  images = []
  labels = []
  for image, label in dataset:
      images.append(image)
      labels.append(label)

  pixel_values = torch.stack(images)
  labels = torch.tensor(labels)
  return {"pixel_values": pixel_values, "labels": labels}

Batch_Size = 16
train_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=Batch_Size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_data, collate_fn=collate_fn, batch_size=Batch_Size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=Batch_Size, shuffle=False, num_workers=2)

In [7]:
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [9]:
from transformers import ViTConfig, ViTForImageClassification

In [10]:
configuration = ViTConfig()
configuration.num_labels = 7
configuration.return_dict = False

In [11]:
model = ViTForImageClassification(configuration).from_pretrained('google/vit-base-patch16-224')
model.num_labels = 7
model.name = 'ViT'
model.classifier = nn.Linear(configuration.hidden_size, configuration.num_labels)
vit = model.to(device)

for param in vit.vit.parameters():
    param.requires_grad = False
for param in vit.classifier.parameters():
    param.requires_grad = True
for param in vit.vit.encoder.layer[-1].parameters():
    param.requires_grad = True
for param in vit.vit.layernorm.parameters():
    param.requires_grad = True

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [12]:
for name, param in vit.named_parameters():
     print(name, param.requires_grad)

vit.embeddings.cls_token False
vit.embeddings.position_embeddings False
vit.embeddings.patch_embeddings.projection.weight False
vit.embeddings.patch_embeddings.projection.bias False
vit.encoder.layer.0.attention.attention.query.weight False
vit.encoder.layer.0.attention.attention.query.bias False
vit.encoder.layer.0.attention.attention.key.weight False
vit.encoder.layer.0.attention.attention.key.bias False
vit.encoder.layer.0.attention.attention.value.weight False
vit.encoder.layer.0.attention.attention.value.bias False
vit.encoder.layer.0.attention.output.dense.weight False
vit.encoder.layer.0.attention.output.dense.bias False
vit.encoder.layer.0.intermediate.dense.weight False
vit.encoder.layer.0.intermediate.dense.bias False
vit.encoder.layer.0.output.dense.weight False
vit.encoder.layer.0.output.dense.bias False
vit.encoder.layer.0.layernorm_before.weight False
vit.encoder.layer.0.layernorm_before.bias False
vit.encoder.layer.0.layernorm_after.weight False
vit.encoder.layer.0.layer

In [13]:
model_parameters = filter(lambda p: p.requires_grad, vit.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

7094791

In [14]:
optimizer = optim.Adam(vit.parameters(), lr = 1e-3)
num_epochs = 30
num_training_steps = num_epochs * len(train_loader)

In [15]:
from tqdm import tqdm
def vit_test_runner(model, dataloader):
  model.eval()

  correct = 0
  total_loss = 0
  total = 0
  for batch in tqdm(dataloader):
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      total_loss += outputs[0]
      logits = outputs[1]
      predictions = torch.argmax(logits, dim=-1)
      total += batch['labels'].size(0)
      correct += (predictions == batch['labels']).sum().item()
  acc = 100 * correct / total
  total_loss = total_loss / total

  return acc, total_loss

In [16]:
def vit_train_runner(model):
  model.train()

  train_loss = 0
  train_correct = 0
  total = 0
  for epoch in range(num_epochs):
    for batch in tqdm(train_loader):
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs[0]
      loss.backward()

      train_loss += loss
      logits = outputs[1]
      predictions = torch.argmax(logits, dim=-1)
      total += batch['labels'].size(0)
      train_correct += (predictions == batch['labels']).sum().item()

      optimizer.step()
      optimizer.zero_grad()
      # torch.cuda.empty_cache()

    train_loss = train_loss / total
    train_acc = 100 * train_correct / total
    print("Epoch {}".format(epoch+1))
    print("Train loss: {}.\t Train accuracy: {:.2f}%".format(train_loss, train_acc))

    valid_acc, valid_loss = vit_test_runner(model, valid_loader)
    print("Valid loss: {}.\t Valid accuracy: {:.2f}%".format(valid_loss, valid_acc))

In [17]:
vit_train_runner(vit)

100%|██████████| 3589/3589 [07:17<00:00,  8.20it/s]


Epoch 1
Train loss: 0.06989163905382156.	 Train accuracy: 57.65%


100%|██████████| 359/359 [00:37<00:00,  9.54it/s]


Valid loss: 0.062485065311193466.	 Valid accuracy: 62.17%


100%|██████████| 3589/3589 [07:12<00:00,  8.30it/s]


...

Valid loss: 0.02922353334724903.	 Valid accuracy: 83.04%


100%|██████████| 3589/3589 [07:13<00:00,  8.28it/s]


Epoch 30
Train loss: 0.0010456532472744584.	 Train accuracy: 73.75%


100%|██████████| 359/359 [00:37<00:00,  9.47it/s]


Valid loss: 0.02834797278046608.	 Valid accuracy: 82.97%


In [18]:
vit_test_runner(vit, test_loader)

100%|██████████| 449/449 [00:47<00:00,  9.46it/s]


(67.13569239342435, tensor(0.0892, device='cuda:0'))

In [19]:
torch.save(vit, '/kaggle/working/vit-ckpt.pt')